In [ ]:
# Мини‑таск 1: Подготовка датасета для анализа активности молекул
# Биологическая мишень: Циклооксигеназа-2 (COX-2) - ChEMBL ID: CHEMBL230

"""Этот notebook выполняет полную обработку данных об активности молекул против COX-2
согласно требованиям задания:
1. Загрузка данных из ChEMBL
2. Отбор релевантных записей
3. Приведение значений активности к единому формату
4. Обработка пропущенных значений и дубликатов
5. Проверка корректности молекул...
"""

import sys

sys.path.append("src")

from pathlib import Path

import polars as pl

from data_processor import COX2DataProcessor
from logging_config import get_logger, setup_logging
from visualization import COX2DataVisualizer

# Настройка логгинга
setup_logging(level="INFO", log_file="./data/cox2_processing.log")
logger = get_logger(__name__)

/home/qwerty/github/datacon2025/.venv/lib/python3.13/site-packages/chembl_webresource_client/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __version__ = __import__('pkg_resources').get_distribution('chembl_webresource_client').version


In [2]:
# Шаг 1: Инициализация обработчика данных
# Создаем процессор для работы с данными COX-2
processor = COX2DataProcessor(target_chembl_id="CHEMBL230", require_pchembl=False)
visualizer = COX2DataVisualizer()

In [3]:
# Шаг 2: Загрузка данных из ChEMBL
logger.info("\nЗагрузка данных об активности IC50 для COX-2 из ChEMBL...")

# Загружаем реальные данные из ChEMBL
raw_data = processor.download_activity_data(activity_type="IC50", limit=None)

logger.info(f"Загружены данные: {raw_data.shape}")
logger.info(f"Колонки: {list(raw_data.columns)}...")

# Просмотр первых записей
logger.info("\nПервые 5 записей загруженных данных:")
logger.info(f"\n{raw_data.head()}")

2025-07-09 21:33:43,811 - __main__ - INFO - 
Загрузка данных об активности IC50 для COX-2 из ChEMBL...
2025-07-09 21:33:43,812 - data_processor - INFO - Loading cached data from data/cache/CHEMBL230_IC50_all_nopchembl.json
2025-07-09 21:33:43,860 - data_processor - INFO - Loaded 7979 cached activity records
2025-07-09 21:33:43,942 - __main__ - INFO - Загружены данные: (7979, 46)
2025-07-09 21:33:43,943 - __main__ - INFO - Колонки: ['action_type', 'activity_comment', 'activity_id', 'activity_properties', 'assay_chembl_id', 'assay_description', 'assay_type', 'assay_variant_accession', 'assay_variant_mutation', 'bao_endpoint', 'bao_format', 'bao_label', 'canonical_smiles', 'data_validity_comment', 'data_validity_description', 'document_chembl_id', 'document_journal', 'document_year', 'ligand_efficiency', 'molecule_chembl_id', 'molecule_pref_name', 'parent_molecule_chembl_id', 'pchembl_value', 'potential_duplicate', 'qudt_units', 'record_id', 'relation', 'src_id', 'standard_flag', 'standar

In [4]:
# Шаг 3: Отбор релевантных записей
logger.info("\n🔍 Отбираем релевантные колонки для анализа...")

# Выбираем только нужные колонки согласно заданию
processed_df = processor.select_relevant_columns(raw_data)

logger.info("\n📋 Отобранные колонки:")
logger.info(f"Размер данных: {processed_df.shape}")
logger.info(f"Колонки: {list(processed_df.columns)}")

# Шаг 4: Приведение значений активности к единому формату (нM)
logger.info("\nПриводим все значения активности к единому формату (nM)...")

processed_df = processor.standardize_activity_units(processed_df)

# Шаг 5: Обработка пропущенных значений и дубликатов
logger.info("\nОбрабатываем пропущенные значения и дубликаты...")

processed_df = processor.clean_missing_values(processed_df)
processed_df = processor.remove_duplicates(processed_df)

logger.info(f"\nПосле очистки: {processed_df.shape}")

# Шаг 6: Проверка корректности молекул
logger.info("\nПроверяем корректность SMILES и рассчитываем молекулярные свойства...")

try:
    processed_df = processor.validate_molecules(processed_df)
    logger.info("Все молекулы валидированы с помощью RDKit")
except Exception as e:
    logger.error(f"Ошибка при валидации молекул: {e}")
    raise

logger.info(f"\nФинальный размер данных: {processed_df.shape}")
logger.info(f"Итоговые колонки: {list(processed_df.columns)}")

2025-07-09 21:33:44,793 - __main__ - INFO - 
🔍 Отбираем релевантные колонки для анализа...
2025-07-09 21:33:44,800 - data_processor - INFO - Column selection: kept 35 of 46 columns (≥20% non-null)
2025-07-09 21:33:44,801 - __main__ - INFO - 
📋 Отобранные колонки:
2025-07-09 21:33:44,802 - __main__ - INFO - Размер данных: (7979, 35)
2025-07-09 21:33:44,802 - __main__ - INFO - Колонки: ['molecule_chembl_id', 'canonical_smiles', 'standard_type', 'standard_value', 'standard_units', 'pchembl_value', 'activity_id', 'activity_properties', 'assay_chembl_id', 'assay_description', 'assay_type', 'bao_endpoint', 'bao_format', 'bao_label', 'document_chembl_id', 'document_journal', 'document_year', 'ligand_efficiency', 'molecule_pref_name', 'parent_molecule_chembl_id', 'potential_duplicate', 'qudt_units', 'record_id', 'relation', 'src_id', 'standard_flag', 'standard_relation', 'target_chembl_id', 'target_organism', 'target_pref_name', 'target_tax_id', 'type', 'units', 'uo_units', 'value']
2025-07-09

Validating molecules: 100%|██████████| 5100/5100 [00:01<00:00, 3597.50it/s]

2025-07-09 21:33:46,340 - data_processor - INFO - Valid molecules: 5100
2025-07-09 21:33:46,340 - data_processor - INFO - Invalid molecules removed: 0
2025-07-09 21:33:46,346 - __main__ - INFO - Все молекулы валидированы с помощью RDKit
2025-07-09 21:33:46,347 - __main__ - INFO - 
Финальный размер данных: (5100, 45)
2025-07-09 21:33:46,347 - __main__ - INFO - Итоговые колонки: ['molecule_chembl_id', 'canonical_smiles', 'standard_type', 'standard_value', 'standard_units', 'pchembl_value', 'activity_id', 'activity_properties', 'assay_chembl_id', 'assay_description', 'assay_type', 'bao_endpoint', 'bao_format', 'bao_label', 'document_chembl_id', 'document_journal', 'document_year', 'ligand_efficiency', 'molecule_pref_name', 'parent_molecule_chembl_id', 'potential_duplicate', 'qudt_units', 'record_id', 'relation', 'src_id', 'standard_flag', 'standard_relation', 'target_chembl_id', 'target_organism', 'target_pref_name', 'target_tax_id', 'type', 'units', 'uo_units', 'value', 'standard_value_n

In [5]:
# Шаг 7: Дополнительная обработка и анализ
logger.info("\nПрименяем дополнительные фильтры и классификацию...")

# Применяем drug-like фильтры для получения более качественного датасета
filtered_df = processor.apply_drug_like_filters(processed_df)

# Добавляем классификацию активности (активные/неактивные соединения)
final_dataset = processor.add_activity_classes(filtered_df)

logger.info(f"\nФинальный датасет после всех фильтров: {final_dataset.shape}")

# Базовая статистика
logger.info("\nАнализ полученного датасета...")

logger.info("\nСтатистика по активности IC50:")
stats_df = final_dataset.select(
    [
        pl.col("standard_value_nm").count().alias("total_compounds"),
        pl.col("standard_value_nm").mean().alias("mean_ic50_nm"),
        pl.col("standard_value_nm").median().alias("median_ic50_nm"),
        pl.col("standard_value_nm").std().alias("std_ic50_nm"),
        pl.col("standard_value_nm").min().alias("min_ic50_nm"),
        pl.col("standard_value_nm").max().alias("max_ic50_nm"),
        pl.col("pic50").mean().alias("mean_pic50"),
    ]
)

logger.info(f"\n{stats_df}")

# Анализ классов активности
logger.info("\nРаспределение по классам активности:")
activity_classes = final_dataset.group_by("activity_class").len().sort("len", descending=True)
logger.info(f"\n{activity_classes}")

# Анализ соответствия drug-like критериям
if "is_drug_like" in final_dataset.columns:
    logger.info("\nСоответствие drug-like критериям:")
    drug_like_stats = final_dataset.group_by("is_drug_like").len()
    logger.info(f"\n{drug_like_stats}")

2025-07-09 21:33:46,351 - __main__ - INFO - 
Применяем дополнительные фильтры и классификацию...
2025-07-09 21:33:46,353 - data_processor - INFO - Applied drug-likeness filters:
2025-07-09 21:33:46,355 - data_processor - INFO -   MW ≤ 500.0  → removed 513
2025-07-09 21:33:46,357 - data_processor - INFO -   logP ≤ 5.0  → removed 1091
2025-07-09 21:33:46,359 - data_processor - INFO -   HBD ≤ 5  → removed 44
2025-07-09 21:33:46,361 - data_processor - INFO -   HBA ≤ 10  → removed 95
2025-07-09 21:33:46,362 - data_processor - INFO -   TPSA ≤ 140.0  → removed 278
2025-07-09 21:33:46,363 - data_processor - INFO -   RotB ≤ 10  → removed 278
2025-07-09 21:33:46,365 - data_processor - INFO -   Activity value > 0  → removed 7
2025-07-09 21:33:46,365 - data_processor - INFO - Total removed: 1439
2025-07-09 21:33:46,365 - data_processor - INFO - Drug-like molecules retained: 3661
2025-07-09 21:33:46,367 - data_processor - INFO - Activity classification summary:
2025-07-09 21:33:46,370 - data_proces

In [6]:
# Шаг 8: Создание визуализаций
logger.info("\nСоздание интерактивных визуализаций...")

# 1. Распределение активности IC50
logger.info("Создание графика распределения активности...")
fig1 = visualizer.plot_activity_distribution(final_dataset, title="Распределение активности IC50 против COX-2")
fig1.show()

# 2. Молекулярные свойства
logger.info("Создание графика молекулярных свойств...")
fig2 = visualizer.plot_molecular_properties(final_dataset, title="Распределение молекулярных свойств")
fig2.show()

# 3. Корреляционная матрица
logger.info("Создание корреляционной матрицы...")
fig3 = visualizer.plot_correlation_matrix(final_dataset, title="Корреляция между молекулярными свойствами")
fig3.show()

# 4. Связь свойств с активностью
logger.info("Создание графика свойства vs активность...")
fig4 = visualizer.plot_property_vs_activity(final_dataset, title="Связь молекулярных свойств с активностью")
fig4.show()

2025-07-09 21:33:47,905 - __main__ - INFO - 
Создание интерактивных визуализаций...
2025-07-09 21:33:47,905 - __main__ - INFO - Создание графика распределения активности...
2025-07-09 21:33:47,905 - visualization - INFO - Creating activity distribution plots...


2025-07-09 21:33:48,090 - __main__ - INFO - Создание графика молекулярных свойств...
2025-07-09 21:33:48,091 - visualization - INFO - Creating molecular properties distribution plots...


2025-07-09 21:33:48,137 - __main__ - INFO - Создание корреляционной матрицы...
2025-07-09 21:33:48,137 - visualization - INFO - Creating correlation matrix plot...


2025-07-09 21:33:48,147 - __main__ - INFO - Создание графика свойства vs активность...
2025-07-09 21:33:48,148 - visualization - INFO - Creating molecular properties vs activity plots...


In [7]:
# Шаг 9: Сохранение финального датасета и генерация отчета
logger.info("\nСохранение обработанного датасета...")

# Сохранение в формате Parquet для максимальной эффективности
output_filename = "cox2_final_dataset.parquet"
processor.save_dataset(final_dataset, filename="cox2_final_dataset.parquet", save_csv=True)

# Генерация и сохранение отчета
logger.info("\nГенерация итогового отчета...")
summary_report = visualizer.generate_summary_report(final_dataset)
logger.info(f"\n{summary_report}")

data_dir = Path("./data")
data_dir.mkdir(exist_ok=True)

with open(data_dir / "cox2_final_report.txt", "w", encoding="utf-8") as f:
    f.write(summary_report)

2025-07-09 21:33:48,421 - __main__ - INFO - 
Сохранение обработанного датасета...
2025-07-09 21:33:48,429 - data_processor - INFO - Dataset saved to data/cox2_final_dataset.parquet
2025-07-09 21:33:48,431 - data_processor - INFO - Flattened nested columns for CSV: ['activity_properties', 'ligand_efficiency']
2025-07-09 21:33:48,457 - data_processor - INFO - Dataset saved to data/cox2_final_dataset.csv (flattened nested columns)
2025-07-09 21:33:48,458 - __main__ - INFO - 
Генерация итогового отчета...
2025-07-09 21:33:48,462 - __main__ - INFO - 
COX-2 DATASET SUMMARY REPORT

DATASET OVERVIEW
Total molecules: 3,661
Dataset shape: (3661, 48)

ACTIVITY STATISTICS
IC50 range: 0.01 - 223342021.07 nM
Median IC50: 1800.00 nM
Mean IC50: 207557.14 nM

ACTIVITY CLASSIFICATION
Low Activity: 2,036 (55.6%)
Moderately Active: 854 (23.3%)
Highly Active: 771 (21.1%)

MOLECULAR PROPERTIES
Average molecular weight: 362.9 Da
Average LogP: 3.44
Average TPSA: 73.1 Ų

DRUG-LIKENESS
Lipinski compliant: 3,661